In [ ]:
# TODO UPDATE THIS OLD MESSY CODE WITH NEW PANEL

In [720]:
import pandas as pd
import datetime as dtime
import numpy as np
import os
import matplotlib.pyplot as plt
import statistics
import math
import scipy
from matplotlib import cm
from scipy.stats import kurtosis
from scipy.stats import skew
from datetime import timedelta

In [721]:
# read in the data
panel_df = pd.read_csv(r"~/Desktop/research/raw_panel.csv",index_col=0)
panel_df['date'] = pd.to_datetime(panel_df['date'])

In [722]:
# data frame for daily frequency
daily_df = panel_df.copy(deep = True)

In [723]:
# create the data frame for weekly frequency where only sundays are kept
def formWeeklyDf(df):
    df['weekday'] = df['date'].dt.day_of_week
    weekly_df = panel_df[panel_df.weekday == 6]
    return weekly_df

In [724]:
# create data frame with monthly frequency where we define months as every 28 days
def formMonthlyDf(weekly_df):
    weekly_df.assign(week_count=weekly_df.groupby(['date']).ngroup() + 1)
    temp_df = weekly_df[['date']].drop_duplicates().reset_index(drop=True)
    temp_df['week_count'] = np.arange(temp_df.shape[0])
    weekly_df = weekly_df.merge(temp_df,
                            on=['date'],
                            how='inner',
                            validate='many_to_one')
    monthly_df = weekly_df[0 == (weekly_df.week_count % 4)]
    return monthly_df

In [725]:
# compute daily returns for each asset
def computeDailyReturn(daily_df):
    temp_daily_df = daily_df.copy()
    temp_daily_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
    temp_daily_df = temp_daily_df[["date","asset","usd_per_token_1"]]
    temp_daily_df["date"] = temp_daily_df["date"] + timedelta(days=1)
    daily_df = temp_daily_df.merge(daily_df,
                               on=["date", "asset"],
                               how='inner',
                               validate='one_to_one')
    daily_df['simpleret'] = (daily_df.usd_per_token-daily_df.usd_per_token_1)/daily_df.usd_per_token_1
    daily_df = daily_df.drop('usd_per_token_1', axis=1)
    return daily_df

In [726]:
# compute the weekly returns for each asset
def computeWeeklyReturn(weekly_df):
    temp_weekly_df = weekly_df.copy()
    temp_weekly_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
    temp_weekly_df = temp_weekly_df[["date","asset","usd_per_token_1"]]
    temp_weekly_df["date"] = temp_weekly_df["date"] + timedelta(days=7)
    weekly_df = temp_weekly_df.merge(weekly_df,
                            on=["date", "asset"],
                            how='inner',
                            validate='one_to_one')
    weekly_df['simpleret'] = (weekly_df.usd_per_token-weekly_df.usd_per_token_1)/weekly_df.usd_per_token_1
    weekly_df = weekly_df.drop('usd_per_token_1', axis=1)
    return weekly_df

In [727]:
# compute the monthly returns for each asset
def computeMonthlyReturn(monthly_df):
    temp_monthly_df = monthly_df.copy()
    temp_monthly_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
    temp_monthly_df = temp_monthly_df[["date","asset","usd_per_token_1"]]
    temp_monthly_df["date"] = temp_monthly_df["date"] + timedelta(days=28)
    monthly_df = temp_monthly_df.merge(monthly_df,
                            on=["date", "asset"],
                            how='inner',
                            validate='one_to_one')
    monthly_df['simpleret'] = (monthly_df.usd_per_token-monthly_df.usd_per_token_1)/monthly_df.usd_per_token_1
    monthly_df = monthly_df.drop('usd_per_token_1', axis=1)
    return monthly_df

In [728]:
# create new data where only data from 2016-2021 are kept
def formNewDf(df):
    df["date"] = pd.to_datetime(df.date)
    new_df = df[df.date.dt.year != 2015]
    return new_df

In [729]:
daily_df = computeDailyReturn(MakeCopy(panel_df))
weekly_df = computeWeeklyReturn(formWeeklyDf(panel_df))
monthly_df = computeMonthlyReturn(formMonthlyDf(weekly_df))

new_daily_df = formNewDf(daily_df)
new_weekly_df = formNewDf(weekly_df)
new_monthly_df = formNewDf(monthly_df)

In [ ]:
# recreate the daily simple returns histogrm alone
def plotIndividualDailyHist(df):
    plt.figure(figsize=(5*1.61, 5))
    plt.xlim(-1,2)
    fig,ax = plt.subplots()
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    daily = plt.hist(df, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2])
    file_path = os.path.join('/Users/chuhanguo','Daily_Returns.jpeg')
    plt.savefig(file_path, bbox_inches='tight')
    return daily
daily = plotIndividualDailyHist(new_daily_df.simpleret)

In [ ]:
# recreate the weekly simple returns histogrm alone (some extremely positive returns (outliers) might not be captured but might affect the distribution)
def plotIndividualWeeklyHist(df):
    plt.figure(figsize=(5*1.61, 5))
    plt.xlim(-1,2)
    fig,ax = plt.subplots()
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    weekly = plt.hist(df, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2])
    file_path = os.path.join('/Users/chuhanguo','Weekly_Returns.jpeg')
    plt.savefig(file_path, bbox_inches='tight')
    return weekly
weekly = plotIndividualWeeklyHist(new_weekly_df.simpleret)

In [ ]:
# recreate the monthly simple returns histogrm alone (some extremely positive returns (outliers) might not be captured but might affect the distribution)
def plotIndividualMonthlyHist(df):
    plt.figure(figsize=(5*1.61, 5))
    plt.xlim(-1,6)
    fig,ax = plt.subplots()
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    monthly = plt.hist(df, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6])
    file_path = os.path.join('/Users/chuhanguo','Monthly_Returns.jpeg')
    plt.savefig(file_path, bbox_inches='tight')
    return monthly
monthly = plotIndividualMonthlyHist(new_monthly_df.simpleret)

In [ ]:
def computeDailyMcapReturn (df):
    # create the within day total market cap for each period
    df['mcap_sum'] = df.groupby('date')['usd_mcap'].transform('sum')
    # compute the market cap share of each asset on each day
    df['mcap_weight'] = df.usd_mcap / df.mcap_sum
    # calculate the within day average return where each asset's return is weighted by its market share
    df['mcap_r_t'] = df.simpleret * df.mcap_weight
    df['mcap_r_t'] = df.groupby('date')[['mcap_r_t']].transform(sum)
    return df
new_daily_df = computeDailyMcapReturn(new_daily_df)

In [ ]:
def computeMcapReturn (df):
    df['mcap_sum'] = df.groupby('date')['usd_mcap'].transform('sum')
    df['mcap_weight'] = df.usd_mcap / df.mcap_sum
    df['mcap_r_t'] = df.simpleret * df.mcap_weight
    df['mcap_r_t'] = df.groupby('date')[['mcap_r_t']].transform(sum)
    df = df[['date','mcap_r_t']].drop_duplicates()
    return df

new_weekly_df = computeMcapReturn(new_weekly_df)
new_monthly_df = computeMcapReturn(new_monthly_df)

In [ ]:
def computeCumProd(new_daily_df):
    new_daily_df["one_plus_simpleret"] = new_daily_df["simpleret"]+1
    new_daily_df["one_plus_mcap_rt"] = new_daily_df["mcap_r_t"]+1

    new_daily_df["daily_val_investment"] = new_daily_df.groupby('date')['one_plus_simpleret'].cumprod() # cumulative product of all daily simple returns

    new_daily_df["bitcoin_return"] = new_daily_df.simpleret[new_daily_df.asset=="bitcoin"]
    new_daily_df["one_plus_bitcoin_return"] = new_daily_df["bitcoin_return"].fillna(0)+1
    new_daily_df["bitcoin_cum_ret"] = new_daily_df.groupby('date')['one_plus_bitcoin_return'].cumprod() # cumulative product of daily simple returns of bitcoin

    new_daily_df["ethereum_return"] = new_daily_df.simpleret[new_daily_df.asset=="ethereum"]
    new_daily_df["one_plus_ethereum_return"] = new_daily_df["ethereum_return"].fillna(0)+1
    new_daily_df["ethereum_cum_ret"] = new_daily_df.groupby('date')['one_plus_ethereum_return'].cumprod() # cumultaive product of daily simple returns of ethereum

    new_daily_df["CMKT_val_investment"] = new_daily_df.groupby('date')['one_plus_mcap_rt'].cumprod()# cumulative product of daily market weighted simple returns
    
    return new_daily_df
new_daily_df = computeCumProd(new_daily_df)

In [736]:
def createAssetDf(name):
    df = new_daily_df[name]
    return df
bitcoin_df = createAssetDf(new_daily_df.asset == "bitcoin")
eth_df = createAssetDf(new_daily_df.asset == "ethereum")

In [737]:
def createCmktDf(df):
    cmkt_df = df[['date','one_plus_mcap_rt']]
    cmkt_df = cmkt_df.drop_duplicates()
    return cmkt_df
cmkt_df = createCmktDf(new_daily_df)

In [738]:
def AssetCumProd(df):
    df_cum = df.cumprod().values
    return df_cum
btc_cum_inv_array = AssetCumProd(bitcoin_df['one_plus_simpleret'])
eth_cum_inv_array = AssetCumProd(eth_df['one_plus_simpleret'])
cmkt_cum_inv_array = AssetCumProd(cmkt_df['one_plus_mcap_rt'])

In [ ]:
cum_inv_df = pd.DataFrame(data={'Bitcoin':btc_cum_inv_array,
                   'Ethereum':eth_cum_inv_array,
                   'Crypto Market':cmkt_cum_inv_array})

def plotTimeSeries(df):
    dates = bitcoin_df.date.values
    df = df.set_index(dates)
    plt.figure(figsize=(5*1.61, 5))
    tsplot = df.plot(style = ['b-','g--','r-.'],cmap='viridis')
    plt.box(False)
    plt.grid(visible=True, which='major', axis='x', linewidth=0.5)
    plt.legend(labels=df.columns.values,
               loc='lower center',
               ncol=3,
               bbox_to_anchor=(0.5, -0.37),
               frameon=False)
    plt.tick_params(axis='both', which='both', bottom=False)
    plt.xticks(rotation=0, ha='center')
    file_path = os.path.join('/Users/chuhanguo','time_series_plot.jpeg')
    plt.savefig(file_path, bbox_inches='tight')
    return tsplot
tsplot = plotTimeSeries(cum_inv_df)

In [ ]:
VTI = pd.read_csv("~/Desktop/research/VTI (1).csv")
VTI_df = pd.DataFrame(data={'Date':VTI['Date'],
                            'Price':VTI['Adj Close']})
VTI_df['Date'] = pd.to_datetime(VTI_df.Date)
VTI_df         = VTI_df[VTI_df.Date.dt.year >= 2016]
VTI_df         = VTI_df[VTI_df.Date.dt.year <= 2021]
VTI_df = VTI_df.reset_index(drop=True)
VTI_df['returns'] = VTI_df['Price'].pct_change(1)
VTI_df = VTI_df.drop(index=0)

VTI_df['weekday'] = \
pd.to_datetime(VTI_df['Date']).dt.day_of_week
VTI_df_weekly = VTI_df[VTI_df.weekday == 0]
VTI_df_weekly['day_diff'] = (pd.to_datetime(VTI_df_weekly.Date)-
                             pd.to_datetime(VTI_df_weekly.Date).shift(1))[1:].astype('timedelta64[D]').astype(int)
VTI_df_weekly = VTI_df_weekly[VTI_df_weekly.day_diff==7]
VTI_df_weekly['weekly_return'] = VTI_df_weekly['Price'].pct_change(1)
new_weekly_VTI_df = VTI_df_weekly.reset_index()[1:]

VTI_df['month_end'] = \
pd.to_datetime(VTI_df.Date).dt.is_month_end
VTI_monthly_df = VTI_df[VTI_df['month_end'] == 1 ]
VTI_monthly_df['monthly_return'] = VTI_monthly_df['Price'].pct_change(1)
new_VTI_monthly_df = VTI_monthly_df.reset_index()[1:]

cmkt_df['mcap_rt'] = cmkt_df['one_plus_mcap_rt']-1


In [ ]:
def createAssetWklyDf(df):
    weekly_df = df[df.date.dt.weekday == 6]
    weekly_df['weekly_return'] = weekly_df['usd_per_token'].pct_change(1)
    new_wkly_df = weekly_df.reset_index()[1:]
    return new_wkly_df
new_weekly_bitcoin_df = createAssetWklyDf(bitcoin_df)
new_weekly_eth_df = createAssetWklyDf(eth_df)

def createAssetMonthlyDf(df):
    df['month_end'] = \
    pd.to_datetime(df.date).dt.is_month_end
    monthly_df = df[df['month_end'] == 1]
    monthly_df['monthly_returns'] = monthly_df['usd_per_token'].pct_change(1)
    new_mthly_df = monthly_df.reset_index()[1:]
    return new_mthly_df
new_monthly_bitcoin_df = createAssetMonthlyDf(bitcoin_df)
new_monthly_eth_df = createAssetMonthlyDf(eth_df)

In [745]:
lenvti = len(VTI_df.returns)
lencmkt = len(cmkt_df.mcap_rt)-2
lenbit = len(bitcoin_df.simpleret)-2
leneth = len(eth_df.simpleret)-2
lenwkvti = len(new_weekly_VTI_df.weekly_return)+1
lenwkcmkt = len(new_weekly_df.mcap_r_t)-1
lenwkbit = len(new_weekly_bitcoin_df.weekly_return)-1
lenwketh = len(new_weekly_eth_df.weekly_return)-1
lenmthvti = len(new_VTI_monthly_df.monthly_return)
lenmthcmkt = len(new_monthly_bitcoin_df.monthly_returns)+1
lenmthbit = len(new_monthly_bitcoin_df.monthly_returns)
lenmtheth = len(new_monthly_eth_df.monthly_returns)

def computeMean(df):
    mean_df = statistics.mean(df)
    return mean_df
VTI_mean = computeMean(VTI_df.returns)
cmkt_mean = computeMean(cmkt_df.mcap_rt)
bitcoin_mean = computeMean(bitcoin_df.simpleret)
ethereum_mean = computeMean(eth_df.simpleret)
VTI_mean_weekly = computeMean(new_weekly_VTI_df.weekly_return)
bitcoin_mean_weekly = computeMean(new_weekly_bitcoin_df.weekly_return)
eth_mean_weekly = computeMean(new_weekly_eth_df.weekly_return)
cmkt_mean_weekly = computeMean(new_weekly_df.mcap_r_t)
VTI_mean_monthly = computeMean(new_VTI_monthly_df.monthly_return)
bitcoin_mean_monthly = computeMean(new_monthly_bitcoin_df.monthly_returns)
eth_mean_monthly = computeMean(new_monthly_eth_df.monthly_returns)
cmkt_mean_monthly = computeMean(new_monthly_df.mcap_r_t)

def computeSd(df):
    sd_df = statistics.stdev(df)
    return sd_df
VTI_sd = computeSd(VTI_df.returns)
cmkt_sd = computeSd(cmkt_df.mcap_rt)
bitcoin_sd = computeSd(bitcoin_df.simpleret)
ethereum_sd = computeSd(eth_df.simpleret)
VTI_sd_weekly = computeSd(new_weekly_VTI_df.weekly_return)
cmkt_sd_weekly = computeSd(new_weekly_df.mcap_r_t)
bitcoin_sd_weekly = computeSd(new_weekly_bitcoin_df.weekly_return)
eth_sd_weekly = computeSd(new_weekly_eth_df.weekly_return)
VTI_sd_monthly = computeSd(new_VTI_monthly_df.monthly_return)
cmkt_sd_monthly = computeSd(new_monthly_df.mcap_r_t)
bitcoin_sd_monthly = computeSd(new_monthly_bitcoin_df.monthly_returns)
eth_sd_monthly = computeSd(new_monthly_eth_df.monthly_returns)

def computeTstat(mean,sd,sqrt):
    t_stat_df = mean/(sd/sqrt)
    return t_stat_df
VTI_t_stat = computeTstat(VTI_mean,VTI_sd,math.sqrt(lenvti))
cmkt_t_stat = computeTstat(cmkt_mean,cmkt_sd,math.sqrt(lencmkt))
bitcoin_t_stat = computeTstat(bitcoin_mean,bitcoin_sd,math.sqrt(lenbit))
ethereum_t_stat = computeTstat(ethereum_mean,ethereum_sd,math.sqrt(leneth))
VTI_t_stat_weekly = computeTstat(VTI_mean_weekly,VTI_sd_weekly,math.sqrt(lenwkvti))
cmkt_t_stat_weekly = computeTstat(cmkt_mean_weekly,cmkt_sd_weekly,math.sqrt(lenwkcmkt))
bitcoin_t_stat_weekly = computeTstat(bitcoin_mean_weekly,bitcoin_sd_weekly,math.sqrt(lenwkbit))
eth_t_stat_weekly = computeTstat(eth_mean_weekly,eth_sd_weekly,math.sqrt(lenwketh))
VTI_t_stat_monthly = computeTstat(VTI_mean_monthly,VTI_sd_monthly,math.sqrt(lenmthvti))
cmkt_t_stat_monthly = computeTstat(cmkt_mean_monthly,cmkt_sd_monthly,math.sqrt(lenmthcmkt))
bitcoin_t_stat_monthly = computeTstat(bitcoin_mean_monthly,bitcoin_sd_monthly,math.sqrt(lenmthbit))
eth_t_stat_monthly = computeTstat(eth_mean_monthly,eth_sd_monthly,math.sqrt(lenmtheth))

def computeSharpe(mean,sd):
    sharpe_df = mean / sd
    return sharpe_df
VTI_sharpe = computeSharpe(VTI_mean,VTI_sd)
cmkt_sharpe = computeSharpe(cmkt_mean,cmkt_sd)
bitcoin_sharpe = computeSharpe(bitcoin_mean,bitcoin_sd)
ethereum_sharpe = computeSharpe(ethereum_mean,ethereum_sd)
VTI_sharpe_weekly = computeSharpe(VTI_mean_weekly,VTI_sd_weekly)
cmkt_sharpe_weekly = computeSharpe(cmkt_mean_weekly,cmkt_sd_weekly)
bitcoin_sharpe_weekly = computeSharpe(bitcoin_mean_weekly,bitcoin_sd_weekly)
eth_sharpe_weekly = computeSharpe(eth_mean_weekly,eth_sd_weekly)
VTI_sharpe_monthly = computeSharpe(VTI_mean_monthly,VTI_sd_monthly)
cmkt_sharpe_monthly = computeSharpe(cmkt_mean_monthly,cmkt_sd_monthly)
bitcoin_sharpe_monthly = computeSharpe(bitcoin_mean_monthly,bitcoin_sd_monthly)
eth_sharpe_monthly = computeSharpe(eth_mean_monthly,eth_sd_monthly)

def computeSkew(df):
    skew_df = skew(df)
    return skew_df
VTI_skewness = computeSkew(VTI_df.returns)
cmkt_skewness = computeSkew(cmkt_df.mcap_rt)
bitcoin_skewness = computeSkew(bitcoin_df.simpleret)
ethereum_skewness = computeSkew(eth_df.simpleret)
VTI_skewness_weekly = computeSkew(new_weekly_VTI_df.weekly_return)
cmkt_skewness_weekly = computeSkew(new_weekly_df.mcap_r_t)
bitcoin_skewness_weekly = computeSkew(new_weekly_bitcoin_df.weekly_return)
eth_skewness_weekly = computeSkew(new_weekly_eth_df.weekly_return)
VTI_skewness_monthly = computeSkew(new_VTI_monthly_df.monthly_return)
cmkt_skewness_monthly = computeSkew(new_monthly_df.mcap_r_t)
bitcoin_skewness_monthly = computeSkew(new_monthly_bitcoin_df.monthly_returns)
eth_skewness_monthly = computeSkew(new_monthly_eth_df.monthly_returns)

def computeKurt(df):
    kurt_df = kurtosis(df,fisher = False)
    return kurt_df
VTI_kurt = computeKurt(VTI_df.returns)
cmkt_kurt = computeKurt(cmkt_df.mcap_rt)
bitcoin_kurt = computeKurt(bitcoin_df.simpleret)
ethereum_kurt = computeKurt(eth_df.simpleret)
VTI_kurt_weekly = computeKurt(new_weekly_VTI_df.weekly_return)
cmkt_kurt_weekly = computeKurt(new_weekly_df.mcap_r_t)
bitcoin_kurt_weekly = computeKurt(new_weekly_bitcoin_df.weekly_return)
eth_kurt_weekly = computeKurt(new_weekly_eth_df.weekly_return)
VTI_kurt_monthly = computeKurt(new_VTI_monthly_df.monthly_return)
cmkt_kurt_monthly = computeKurt(new_monthly_df.mcap_r_t)
bitcoin_kurt_monthly = computeKurt(new_monthly_bitcoin_df.monthly_returns)
eth_kurt_monthly = computeKurt(new_monthly_eth_df.monthly_returns)

def computePercentPositive(df,ret,num):
    pos_df = len(df.loc[ret>0])/num
    return pos_df
VTI_positive = computePercentPositive(VTI_df,VTI_df.returns,lenvti)
cmkt_positive = computePercentPositive(cmkt_df,cmkt_df.mcap_rt,lencmkt)
bitcoin_positive = computePercentPositive(bitcoin_df,bitcoin_df.simpleret,lenbit)
ethereum_positive = computePercentPositive(eth_df,eth_df.simpleret,leneth)
VTI_positive_weekly = computePercentPositive(new_weekly_VTI_df,new_weekly_VTI_df.weekly_return,lenwkvti)
cmkt_positive_weekly = computePercentPositive(new_weekly_df,new_weekly_df.mcap_r_t,lenwkcmkt)
bitcoin_positive_weekly = computePercentPositive(new_weekly_bitcoin_df,new_weekly_bitcoin_df.weekly_return,lenwkbit)
eth_positive_weekly = computePercentPositive(new_weekly_eth_df,new_weekly_eth_df.weekly_return,lenwketh)
VTI_positive_monthly = computePercentPositive(new_VTI_monthly_df,new_VTI_monthly_df.monthly_return,lenmthvti)
cmkt_positive_monthly = computePercentPositive(new_monthly_df,new_monthly_df.mcap_r_t,lenmthcmkt)
bitcoin_positive_monthly = computePercentPositive(new_monthly_bitcoin_df,new_monthly_bitcoin_df.monthly_returns,lenmthbit)
eth_positive_monthly = computePercentPositive(new_monthly_eth_df,new_monthly_eth_df.monthly_returns,lenmtheth)

In [746]:
def computeExtr(num):
    dis = len(cmkt_df.loc[cmkt_df.mcap_rt<-num])
    mir = len(cmkt_df.loc[cmkt_df.mcap_rt>num])
    return dis,mir
cmkt_disaster_5,cmkt_miracle_5 = computeExtr(0.05)                                      
cmkt_disaster_10,cmkt_miracle_10 = computeExtr(0.10)
cmkt_disaster_15,cmkt_miracle_15 = computeExtr(0.15)
cmkt_disaster_20,cmkt_miracle_20 = computeExtr(0.20)    

def computeExtrPer(dt):
    per = dt/lencmkt
    return per
cmkt_disaster_5_percent = computeExtrPer(cmkt_disaster_5)
cmkt_disaster_10_percent = computeExtrPer(cmkt_disaster_10)
cmkt_disaster_15_percent = computeExtrPer(cmkt_disaster_15)
cmkt_disaster_20_percent = computeExtrPer(cmkt_disaster_20)
cmkt_miracle_5_percent = computeExtrPer(cmkt_miracle_5)
cmkt_miracle_10_percent = computeExtrPer(cmkt_miracle_10)
cmkt_miracle_15_percent = computeExtrPer(cmkt_miracle_15)
cmkt_miracle_20_percent = computeExtrPer(cmkt_miracle_20)

In [747]:
Extreme_events_df = pd.DataFrame(data={"Disasters":("< -5%","< -10%","< -15%","< -20%"),
                                      "Disaster_Counts":(cmkt_disaster_5,cmkt_disaster_10,cmkt_disaster_15,cmkt_disaster_20),
                                      "Disaster_Proportion":(cmkt_disaster_5_percent,cmkt_disaster_10_percent,cmkt_disaster_15_percent,cmkt_disaster_20_percent),
                                      "Miracles":("> 5%","> 10%","> 15%","> 20%"),
                                      "Miracle_Counts":(cmkt_miracle_5,cmkt_miracle_10,cmkt_miracle_15,cmkt_miracle_20),
                                      "Miracle_Proportions":(cmkt_miracle_5_percent,cmkt_miracle_10_percent,cmkt_miracle_15_percent,cmkt_miracle_20_percent)})

In [748]:
daily_stats_df = pd.DataFrame(data={"daily":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean,bitcoin_mean,ethereum_mean,VTI_mean),
                                  "sd":(cmkt_sd,bitcoin_sd,ethereum_sd,VTI_sd),
                                  "t_stat":(cmkt_t_stat,bitcoin_t_stat,ethereum_t_stat,VTI_t_stat),
                                  "sharpe":(cmkt_sharpe,bitcoin_sharpe,ethereum_sharpe,VTI_sharpe),
                                  "skewness":(cmkt_skewness,bitcoin_skewness,ethereum_skewness,VTI_skewness),
                                  "kurtosis":(cmkt_kurt,bitcoin_kurt,ethereum_kurt,VTI_kurt),
                                  "% > 0":(cmkt_positive,bitcoin_positive,ethereum_positive,VTI_positive)
                                  })

In [749]:
weekly_stats_df = pd.DataFrame(data={"weekly":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean_weekly,bitcoin_mean_weekly,eth_mean_weekly,VTI_mean_weekly),
                                    "sd":(cmkt_sd_weekly,bitcoin_sd_weekly,eth_sd_weekly,VTI_sd_weekly),
                                    "t_stat":(cmkt_t_stat_weekly,bitcoin_t_stat_weekly,eth_t_stat_weekly,VTI_t_stat_weekly),
                                    "sharpe":(cmkt_sharpe_weekly,bitcoin_sharpe_weekly,eth_sharpe_weekly,VTI_sharpe_weekly),
                                    "skewness":(cmkt_skewness_weekly,bitcoin_skewness_weekly,eth_skewness_weekly,VTI_skewness_weekly),
                                    "kurtosis":(cmkt_kurt_weekly,bitcoin_kurt_weekly,eth_kurt_weekly,VTI_kurt_weekly),
                                    "% > 0":(cmkt_positive_weekly,bitcoin_positive_weekly,eth_positive_weekly,VTI_positive_weekly)})

In [750]:
# MIGHT NEED TO TAKE A LOOK INTO THIS as there are some month-end dates that falls in weekends / holidays and not sure if it will cause a material change to the descriptive statistics of the VTI stock.
monthly_stats_df = pd.DataFrame(data={"monthly":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean_monthly,bitcoin_mean_monthly,eth_mean_monthly,VTI_mean_monthly),
                                    "sd":(cmkt_sd_monthly,bitcoin_sd_monthly,eth_sd_monthly,VTI_sd_monthly),
                                    "t_stat":(cmkt_t_stat_monthly,bitcoin_t_stat_monthly,eth_t_stat_monthly,VTI_t_stat_monthly),
                                    "sharpe":(cmkt_sharpe_monthly,bitcoin_sharpe_monthly,eth_sharpe_monthly,VTI_sharpe_monthly),
                                    "skewness":(cmkt_skewness_monthly,bitcoin_skewness_monthly,eth_skewness_monthly,VTI_skewness_monthly),
                                    "kurtosis":(cmkt_kurt_monthly,bitcoin_kurt_monthly,eth_kurt_monthly,VTI_kurt_monthly),
                                      "% > 0":(cmkt_positive_monthly,bitcoin_positive_monthly,eth_positive_monthly,VTI_positive_monthly)})

In [752]:
fp = '~/desktop/research/descriptive_stats.xlsx'
with pd.ExcelWriter(fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
    daily_stats_df.to_excel(writer,sheet_name='daily')
    weekly_stats_df.to_excel(writer,sheet_name='weekly')
    monthly_stats_df.to_excel(writer,sheet_name='monthly')
    Extreme_events_df.to_excel(writer,sheet_name='extreme')